In [20]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import os
import time
from selenium.webdriver.common.by import By

df = pd.read_csv('./루리웹__게임 리스트.csv')
review_list = df.iloc[:,[2]].values.tolist()


review_data = pd.DataFrame({'게임명':[], "리뷰":[], "추천":[], "비추천":[], "조회":[], "날짜":[]})


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

driver.implicitly_wait(10)

<ipython-input-20-083c3e1b2cab>:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')


In [3]:
df.head()

,Unnamed: 0,Name(Eng),Name(Kor),Release date
0,NaN,Rain World,레인 월드,2017-03-27
1,NaN,Kero Blaster,케로 블래스터,2014-05-11
2,NaN,Valkyrie Drive: Bhikkhuni,발키리 드라이브 비쿠니,2015-12-10
3,NaN,Headmaster,헤드마스터,2016-10-13
4,NaN,NEON STRUCT,네온 스트럭트,2015-04-03


In [24]:
iwantreview = ["후기", "리뷰"]

num = 0
for review in review_list:
    review = review[0]
    # 루리웹 게임찾기 페이지
    driver.implicitly_wait(1)
    driver.get('https://bbs.ruliweb.com/game')
    time.sleep(2)
    # 게임 리스트 검색
    driver.find_element("xpath",'//*[@id="game_search"]/form/div[1]/div[2]/div/div[4]/div[1]/div/input[5]').send_keys(review)
    driver.find_element("xpath",'//*[@id="game_search"]/form/div[1]/div[2]/div/div[4]/div[1]/div/button').click()
    time.sleep(1)

    try:
        driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/p') # 검색 결과없음 창이 뜬다면?
        print('게임 데이터가 없습니다.')
        driver.get('https://bbs.ruliweb.com/game') # 다시 게임찾기 페이지로
    except:
        driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/ul/a/li[1]/strong').click()
        time.sleep(1)
        # '후기' 검색
        driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/input').send_keys(iwantreview[0])
        driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/button').click()
        time.sleep(1)

        if driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/tbody/tr[2]/td/p'):# 만약 게시글이 없을 경우 패스
            print('후기 게시물이 없습니다.')
            pass
        else:
            driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/thead/tr/th[5]/a').click() # 게시글이 있을 경우 추천수 내림차순

        # 대충 개별 후기 들어가서 스크랩하는 코드
        # 
        driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/ul/a/li[1]/strong').click()
        time.sleep(1)
        driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/input').send_keys(iwantreview[1])
        driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/button').click()
        time.sleep(1)

    # else:
    #     driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/p')



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="game_search"]/form/ul/li/div/div[2]/div/ul/a/li[1]/strong"}
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x0097C0A3+2212003]
	Ordinal0 [0x00912CC1+1780929]
	Ordinal0 [0x0082465D+804445]
	Ordinal0 [0x00853475+996469]
	Ordinal0 [0x0085363B+996923]
	Ordinal0 [0x00881382+1184642]
	Ordinal0 [0x0086EC64+1109092]
	Ordinal0 [0x0087F5B2+1177010]
	Ordinal0 [0x0086EA36+1108534]
	Ordinal0 [0x008483C9+951241]
	Ordinal0 [0x00849396+955286]
	GetHandleVerifier [0x00C29CE2+2746722]
	GetHandleVerifier [0x00C1A234+2682548]
	GetHandleVerifier [0x00A0B34A+524234]
	GetHandleVerifier [0x00A09B60+518112]
	Ordinal0 [0x00919FBC+1810364]
	Ordinal0 [0x0091EA28+1829416]
	Ordinal0 [0x0091EB15+1829653]
	Ordinal0 [0x00928744+1869636]
	BaseThreadInitThunk [0x7676FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77E47A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77E47A6E+238]
